# Training Practice Titanic Data

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/Users/lilymartin/DEVOPS/testing_python_docker/app/data/train.csv')

In [77]:
def remove_columns(df, columns = ['Cabin', 'Name', 'Ticket']):
    """
    remove unwanted columns
    """
    cols = [x for x in list(df.columns) if x not in columns]
    return df[cols]

In [78]:
def check_for_nan(df):
    """
    Checking df for any rows with NaN
    """
    return df.dropna()

In [79]:
def age_buckets(df):
    """
    Sort age column into age buckets
    """
    age_col = 'Age'
    df.loc[df[age_col] <= 17, 'AgeBuckets'] = '0_17'
    df.loc[(df[age_col] > 17) & (df[age_col] <= 24), 'AgeBuckets'] = '18_24'
    df.loc[(df[age_col] >= 25) & (df[age_col] <= 34), 'AgeBuckets'] = '25_34'
    df.loc[(df[age_col] >= 35) & (df[age_col] <= 44), 'AgeBuckets'] = '35_44'
    df.loc[(df[age_col] >= 45) & (df[age_col] <= 54), 'AgeBuckets'] = '45_54'
    df.loc[(df[age_col] >= 55) & (df[age_col] <= 64), 'AgeBuckets'] = '55_64'
    df.loc[(df[age_col] >= 65) & (df[age_col] <= 74), 'AgeBuckets'] = '65_74'
    df.loc[(df[age_col] >= 75), 'AgeBuckets'] = '75_plus'
    return df 

In [80]:
def fare_buckets(df):
    """
    Sort fares into buckets
    """
    fare_col = 'Fare'
    df.loc[df[fare_col] <= 9, 'FareBucket'] = '0_9'
    df.loc[(df[fare_col] > 10) & (df[fare_col] <= 16), 'FareBucket'] = '10_16'
    df.loc[(df[fare_col] >= 17) & (df[fare_col] <= 33), 'FareBucket'] = '17_33'
    df.loc[(df[fare_col] >= 34), 'FareBucket'] = '34_plus'
    return df 

In [81]:
def hot_encoding(df):
    """
    Hot encoding spefic columns 
    """
    cols = [x for x in list(df.columns) if x in ['Sex', 'Embarked', 'AgeBuckets', 'FareBucket']]
    
    for x in cols: 
        dummies = pd.get_dummies(df[x], x)
        df = pd.concat([df.drop(x, axis=1), dummies], axis=1)
    return df

In [104]:
# Cleaning 
df = pd.read_csv('/Users/lilymartin/DEVOPS/testing_python_docker/app/data/train.csv')
df = remove_columns(df, ['Cabin', 'Name', 'Ticket'])
df = check_for_nan(df)
df = age_buckets(df)
df = fare_buckets(df)
df = hot_encoding(df)
df = remove_columns(df, ['Age', 'Fare', 'PassengerId'])

### Training

In [111]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [116]:
model = Sequential()
model.add(Dense(12, input_dim=20, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [114]:
X = np.array(df[['Pclass', 'SibSp', 'Parch', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'AgeBuckets_0_17',
       'AgeBuckets_18_24', 'AgeBuckets_25_34', 'AgeBuckets_35_44',
       'AgeBuckets_45_54', 'AgeBuckets_55_64', 'AgeBuckets_65_74',
       'AgeBuckets_75_plus', 'FareBucket_0_9', 'FareBucket_10_16',
       'FareBucket_17_33', 'FareBucket_34_plus']])
Y = np.array(df[['Survived']])

In [117]:
# fit the keras model on the dataset
model.fit(X, Y, epochs=150, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X, Y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/150
72/72 [==============================] - 0s 853us/step - loss: 0.6851 - accuracy: 0.5787
Epoch 2/150
72/72 [==============================] - 0s 897us/step - loss: 0.6030 - accuracy: 0.6784
Epoch 3/150
72/72 [==============================] - 0s 896us/step - loss: 0.5632 - accuracy: 0.7289
Epoch 4/150
72/72 [==============================] - 0s 871us/step - loss: 0.5348 - accuracy: 0.7725
Epoch 5/150
72/72 [==============================] - 0s 1ms/step - loss: 0.5087 - accuracy: 0.7837
Epoch 6/150
72/72 [==============================] - 0s 1ms/step - loss: 0.4940 - accuracy: 0.7823
Epoch 7/150
72/72 [==============================] - 0s 1ms/step - loss: 0.4812 - accuracy: 0.7851
Epoch 8/150
72/72 [==============================] - 0s 945us/step - loss: 0.4748 - accuracy: 0.7865
Epoch 9/150
72/72 [==============================] - 0s 988us/step - loss: 0.4674 - accuracy: 0.7823
Epoch 10/150
72/72 [==============================] - 0s 967us/step - loss: 0.4607 - accuracy: 0.

In [118]:
model_path = '/Users/lilymartin/DEVOPS/testing_python_docker/app/model/20200823_titantic_m1.h5'
model.save(model_path)